In [1]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.autograd import Variable


In [2]:
def unpickle(file):
    import pickle
    with open(file) as fo:
        dict = pickle.load(fo, encoding-'bytes')
    return dict

In [3]:
# Define a convolutional block
# A single layer of convolution if shortcut == False, and...
# Two layers of convolution and a residual convolution otherwise.

class ConvBlock(nn.Module):
    
    # midChannels will be of no use if shortcut == False
    def __init__(self, inChannels, midChannels, outChannels, 
                 kernelSize, stride = 1, padding = 0, bias = True, shortcut = False):
        super(ConvBlock, self).__init__()
        if shortcut is False:
            self.left = nn.Sequential(
                nn.Conv2d(inChannels, outChannels, 
                          kernelSize, stride, padding, bias),
                nn.BatchNorm2d(outChannels)
            )
            self.right = None
        else:
            self.left = nn.Sequential(
                nn.Conv2d(inChannels, midChannels, 
                          kernelSize, stride, padding, bias),
                nn.BatchNorm2d(midChannels),
                nn.ReLU(inplace = True),
                nn.Conv2d(midChannels, outChannels, 
                          kernelSize, stride, padding, bias),
                nn.BatchNorm2d(outChannels)
            )
            self.right = nn.Sequential(
                nn.Conv2d(inChannels, outChannels, 
                          kernelSize, stride, padding, bias),
                nn.BatchNorm2d(outChannels)
            )
        
    def forward(self, input):
        out = self.left(input)
        if self.right is not None:
            out += self.right(input)
        return F.relu(out)
    

In [4]:
# Define a fully connected block
# A single layer if shortcut == False, and...
# Two layers and a residual layer otherwise.

class FCBlock(nn.Module):
    
    # midChannels will be of no use if shortcut == False
    def __init__(self, inNodes, midNodes, outNodes, shortcut = False):
        super(FCBlock, self).__init__()
        if shortcut is False:
            self.left = nn.Sequential(
                nn.Linear(inNodes, outNodes)
            )
            self.right = None
        else:
            self.left = nn.Sequential(
                nn.Linear(inNodes, midNodes),
                nn.ReLU(inplace = True),
                nn.Linear(midNodes, outNodes)
            )
            self.right = nn.Sequential(
                nn.Linear(inNodes, outNodes)
            )
            
    def forward(self, input):
        out = self.left(input)
        if self.right is not None:
            out += self.right(input)
        return F.relu(out)
        

In [5]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [6]:
class ResNet(nn.Module):
    
    def __init__(self):
        super(ResNet, self).__init__()
        
        self.blocks = nn.Sequential(
            ConvBlock(inChannels = 3, midChannels = 8, outChannels = 32, 
                      kernelSize = 3, padding = 1, shortcut = True),
            ConvBlock(inChannels = 32, midChannels = 128, outChannels = 512, 
                      kernelSize = 3, padding = 1, shortcut = True),
            
            Flatten(),
            
            FCBlock(inNodes = 524288, midNodes = 16384, outNodes = 1024, 
                    shortcut = True),
            FCBlock(inNodes = 1024, midNodes = 256, outNodes = 64, 
                    shortcut = True),
            
            FCBlock(inNodes = 64, midNodes = 0, outNodes = 10, 
                    shortcut = True)
            
        )
    
    def forward(self, input):
        out = self.blocks(input)
        return out

In [7]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

numTrain = 49000
numVal = 1000

trainData = datasets.CIFAR10('./data', train = True,
                           transform = transforms.ToTensor())
trainLoader = DataLoader(trainData, batch_size = 64, 
                              sampler = ChunkSampler(numTrain, 0))

valData = datasets.CIFAR10('./data', train = True,
                           transform = transforms.ToTensor())
valLoader = DataLoader(valData, batch_size = 64, 
                            sampler = ChunkSampler(numVal, numTrain))

testData = datasets.CIFAR10('./data', train = False,
                          transform = transforms.ToTensor())
testLoader = DataLoader(testData, batch_size=64)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [8]:
def train(model, lossFunc, optimizer, numEpochs = 1, printEvery = 100):
    for epoch in range(numEpochs):
        print('Starting epoch %d / %d' % (epoch + 1, numEpochs))
        model.train()
        for t, (x, y) in enumerate(trainLoader):
            xVar = Variable(x)
            yVar = Variable(y.long())
            scores = model(xVar)
            
            loss = lossFunc(scores, yVar)
            if (t + 1) % printEvery == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    numCorrect = 0
    numSamples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        xVar = Variable(x, volatile=True)

        scores = model(xVar)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(numCorrect) / numSamples
    print('Got %d / %d correct (%.2f)' % (numCorrect, numSamples, 100 * acc))

In [9]:
model = ResNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

train(model, criterion, optimizer, numEpochs = 5)
check_accuracy(model, valLoader)

Starting epoch 1 / 5


RuntimeError: size mismatch, m1: [64 x 524288], m2: [512 x 256] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:940

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainLoader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

model = ResNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

numEpochs = 5
for epoch in range(numEpochs):
    
    runningLoss = 0.0
    for i, data in enumerate(trainLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, runningLoss / 2000))
            runningLoss = 0.0

main()